<center>

## MAGICIAN: Mining and gAmblinG servIces sCrapIng and ANalysis

Matteo Martini 636694

In [2]:
mining_pool_url = "https://en.bitcoin.it/wiki/DeepBit9"


#Individuare nel DataSet tutte le transazioni relative a Deepbit.net,ovvero le transazioni che presentano almeno un indirizzo di input o di output appartenente a Deepbit_addrs.
#servizio scommesse = DiceOnCrack

## Setup Ambiente

In [3]:
# Librerie concesse
import pandas as pd

# Per scraping (scegline uno o entrambi, in base alle esigenze delle pagine)
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Per formati efficienti (PyArrow)
import pyarrow as pa
import pyarrow.parquet as pq

# Utility
from pathlib import Path
from datetime import datetime

import numpy as np


from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
import time
import requests
import re
from bs4 import BeautifulSoup

# Percorsi (aggiorna con il tuo percorso locale)
DATA_DIR = Path('./Datasets')   # metti qui i CSV scaricati

PARQUET_DIR = Path('./data_parquet')
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
print(DATA_DIR)
print(PARQUET_DIR)
print('Environment ready.')

Datasets
data_parquet
Environment ready.


## 1) Costruzione dei Dataframe dal Dataset

## Directory Datasets


In [4]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

## Definizone Data Type
Sapendo cosa contengono i file csv, possiamo forzare la conversione in determiinati tipi per ridurre l'impatto in memoria dei dataset

In [5]:
dtype_transactions = {
    "timestamp": np.int32,          # prima colonna: secondi (1231469xxx) rientra in int32
    "tx_id": np.int32,              # seconda colonna: contatore 1…10 572 827
    "block_height": np.int32,       # terza colonna: 1…214 562
    "prev_tx_id": np.int32,         # quarta colonna: seguono il tx_id precedente
    "has_witness": np.uint8,        # quinta colonna 0/1
    "fee": np.int32,                # sesta colonna: 0…100 000 (int32 più che sufficiente)
}


dtype_inputs = {
    "input_id": np.int32,
    "tx_id": np.int32,
    "prev_output_id": np.int32,
    "input_index": np.uint8,
}


dtype_outputs = {
    "col0":np.int32,
    "col1":np.int32,
    "col2":np.int8,
    "col3":np.int32,
    "col4":np.int64,
    "col5":"uint8"
}

dtype_mapping = {
    "address": "category",   # stringhe ripetute → category riduce drasticamente
    "address_id": "Int32",   # valori 0–8 708 820 entrano in 32 bit
}

columns_transactions = [
    "timestamp", "tx_id", "block_height",
     "has_witness", "fee"
]

## Caricamento dei Dataset

In [6]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

transactions = pd.read_csv(transactions_csv,header=None,names=columns_transactions,dtype=dtype_transactions)
inputs_df    = pd.read_csv(inputs_csv,dtype=dtype_inputs)
outputs_df   = pd.read_csv(outputs_csv,dtype=dtype_outputs)
mapping_df   = pd.read_csv(mapping_csv,dtype=dtype_mapping)

## 2) Recupero informazioni da WalletExplorer


### Definizione Funzioni Helper per Scraping

In [7]:

#placeholder
wallet_explorer_url = "https://www.walletexplorer.com/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/127.0.0.1 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.imdb.com/",
}

In [ ]:
def next_page(driver):
    #cerco il link alla prossima pagina
    for _ in ("Next","Last"):
        url = find_next_link(driver,_)
        if url:return url
    return None
        
def find_next_link(driver,element):
    try:
        el = driver.find_element(By.XPATH, f'//div[@class="paging"]//a[contains(., "{element}")]')
        url = el.get_attribute("href")
        el.click()
        return url
    except:
        return None

def get_transactions_link(soup,class_):
    transactions = soup.find_all('tr',class_=class_)
    page_links = []
    for trans in transactions:
        txid = trans.find('td',class_="txid").find('a')['href']
        page_links.append(txid)
    return page_links

def safe_get(session, url, tries=10, delay=1):
    for attempt in range(tries):
        try:
            return session.get(url, timeout=10)
        except (requests.exceptions.ConnectionError,
                requests.exceptions.RemoteDisconnectedError):
            if attempt == tries - 1:
                raise
            time.sleep(delay * (attempt + 1))  # 1s, poi 2s, poi 3s…

def scrape_step(driver,url,session):
    res = session.get(url,timeout=10)
    #res = safe_get(session,url)
    soup = BeautifulSoup(res.text,"html.parser")
    #recupero le entries
    global_trans = get_transactions_link(soup,'sent') + get_transactions_link(soup,'received')
    #vado alla prossima pagina
    url = next_page(driver)
    return [url,global_trans]

### Definizione Funzione per Scraping

In [9]:
def scrape_wallet_explorer(wallet_name):
    driver = webdriver.Chrome()
    driver.get(wallet_explorer_url)
    
    #ricerca del wallet
    driver.find_element(By.NAME,'q').send_keys(wallet_name)
    driver.find_element(By.TAG_NAME,'form').submit()
    time.sleep(1)
    url = driver.current_url
    all_entries = []
    session = requests.Session()
    session.headers.update(headers)
    while True:
        #recupero di tutte le transazioni della pagina
        url,page_entries = scrape_step(driver,url,session)
        all_entries.extend(page_entries)
        if url == None:
            break;
    driver.close()
    return all_entries

#https://www.walletexplorer.com/txid/d71d3b18df328681c4e0e4ed215a19820c7ecff956e1b6961eec10b7b3417917


In [10]:
BLOCK_RE = re.compile(r"\d+")
FEE_RE = re.compile(r"\d+(?:\.\d+)?")

def get_info_from_transaction_table(trans_table_link,session):
    # res = session.get(f'{"https://www.walletexplorer.com"}{trans_table_link}',timeout=10)
    res = safe_get(session,f'{"https://www.walletexplorer.com"}{trans_table_link}')
    soup = BeautifulSoup(res.text,"html.parser")
    cells = soup.select('table.info td')
    all_txt = [td.get_text() for td in cells]
    match  = BLOCK_RE.search(all_txt[1])

    match_2 = FEE_RE.search(all_txt[4])
    return {
        "timestamp":all_txt[2],
        "blockId":int(match.group()) if match else None,
        "txId":all_txt[0],
        "isCoinbase":0,
        "fee":float(match_2.group()) if match_2 else None
    }

In [12]:
diceoncrack_transaction = scrape_wallet_explorer('DiceonCrack')
#BW.com

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
def get_all_trans(transaction_list):
    infos = []
    session = requests.Session()
    session.headers.update(headers)
    for transaction_link in transaction_list:
        info = get_info_from_transaction_table(transaction_link,session)
        infos.append(info)
    return infos

In [ ]:
doc_trans = get_all_trans(diceoncrack_transaction[:1000])

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def worker(transactions):
    session = requests.Session()
    session.headers.update(headers)
    return get_info_from_transaction_table(transactions,session)
with ThreadPoolExecutor(max_workers=8) as pool:
    infos = list(pool.map(worker,diceoncrack_transaction[:100]))

NameError: name 'diceoncrack_transaction' is not defined

In [ ]:
display(doc_trans)
print(len(doc_trans))
print(len(diceoncrack_transaction))

[{'timestamp': '2019-09-09 16:23:52',
  'blockId': 594029,
  'txId': '910418f1a4fd923d27472603cea0bcade1653cc295f2840869abb337b3822efd',
  'isCoinbase': 0,
  'fee': 5.73e-06},
 {'timestamp': '2019-09-09 16:19:55',
  'blockId': 594028,
  'txId': 'de7c7a195a01ac4518f10202c4d745fbe7cb53a535c785066bd0841176ae0fc1',
  'isCoinbase': 0,
  'fee': 6.65e-06},
 {'timestamp': '2015-03-15 20:08:19',
  'blockId': 347765,
  'txId': '6f07bbf90e6276a427ed03e11c00e71d455b0d0d408205af4b008687fc5cd601',
  'isCoinbase': 0,
  'fee': 0.001},
 {'timestamp': '2015-03-14 13:59:21',
  'blockId': 347587,
  'txId': '1331071843539f0eef3ea522236b04da5002249a8ac7865a782110f02cce8507',
  'isCoinbase': 0,
  'fee': 0.0019},
 {'timestamp': '2015-03-14 13:59:21',
  'blockId': 347587,
  'txId': 'b6f8f614747236ad5745da4641e5fe9d24cdbf78f5ebab0754df7f3035422d3d',
  'isCoinbase': 0,
  'fee': 0.0003},
 {'timestamp': '2015-02-08 11:14:42',
  'blockId': 342542,
  'txId': 'a39121da37055b5c73e2c2611ca53964506d7baf7f83ced3e4ffaa1d9

1000
91046


In [ ]:
deepbit_transactions = scrape_wallet_explorer('DeepBit.net')